In [ ]:
from classiq import QuantumProgram
from typing import Dict
import numpy as np
from classiq import *
from stateprep_qet.utils import amp_to_prob, find_angle, normalize
from classiq.execution import ClassiqBackendPreferences, ExecutionPreferences
from classiq.qmod.symbolic import sin, cos

NUM_QUBITS = 3  # U is applied to these qubits
POLY_DEGREE = 5
POLY_FUNC = lambda x: x
POLY_MAX_SCALE = 1


@qfunc
def projector_cnot(reg: QNum, aux: QBit) -> None:
    control(reg == 0, lambda: X(aux))


@qfunc
def u_sin(x: QNum, a: QNum) -> None:
    a *= sin(x / (2**NUM_QUBITS))  # Amplitude encoding sin(x) to |1>
    X(a)  # sin(x) to |0>


@qfunc
def main(x: Output[QNum], ind: Output[QNum], aux: Output[QBit]):
    allocate(NUM_QUBITS, x)
    allocate(1, ind)
    allocate(1, aux)

    # # Construct equal superposition of all states
    # hadamard_transform(x)
    u_sin(x, ind)

qmod = create_model(
    main,
    constraints=Constraints(max_width=100),
    execution_preferences=ExecutionPreferences(
        num_shots=1,
        backend_preferences=ClassiqBackendPreferences(
            backend_name="simulator_statevector"
        ),
    ),
    out_file="qsvt",
)



def get_metrics(qprog):
    """
    Extract circuit metrics from a quantum program.

    Parameters:
        qprog: The quantum program object.

    Returns:
        dict: A dictionary containing the circuit metrics:
              - "depth": Circuit depth.
              - "width": Circuit width (number of qubits used).
              - "cx_count": Number of CX gates (returns 0 if none are present).
    """
    # Generate the optimized quantum circuit
    circuit = QuantumProgram.from_qprog(qprog)

    # Extract metrics
    metrics = {
        "depth": circuit.transpiled_circuit.depth,
        "width": circuit.data.width,
        "cx_count": circuit.transpiled_circuit.count_ops.get('cx', 0)  # Default to 0 if 'cx' not found
    }

    return metrics

# Print circuit metrics
qprog = synthesize(qmod)
print(get_metrics(qprog))

{'depth': 12, 'width': 5, 'cx_count': 8}
